In [1]:
import pandas as pd
import os 

In [2]:
input_data_dir = os.getcwd()+"\..\datasets\\"

In [5]:
CA_df = pd.read_csv(input_data_dir + "California_Agewise_Data.csv")

In [6]:
CA_df = CA_df.replace(to_replace="65 and Older",value="65+")

In [7]:
CA_df['date']= pd.to_datetime(CA_df['date']) 
age_groups = list(set(CA_df['age_group'].values))
age_groups.remove("Missing")
age_groups.remove("Unknown")
age_groups.sort()

In [8]:
age_groups

['0-17', '18-49', '50-64', '65+']

In [9]:
count = 0
for age_group in age_groups:
    
    CA_df_age_group = CA_df[CA_df['age_group'] == age_group]
    CA_df_age_group = CA_df_age_group.drop(['age_group','deaths','case_percent','deaths_percent','ca_percent'],axis=1)
    CA_df_age_group = CA_df_age_group.rename(columns={'totalpositive':'Infected_'+age_group})
    CA_df_age_group.set_index('date', inplace=True)
    
    print(age_group,len(CA_df_age_group))
    if(count==0):
        CA_df_final = CA_df_age_group
        count+=1
    else:
        CA_df_final = CA_df_final.merge(CA_df_age_group,on='date', how='outer')

CA_df_final = CA_df_final.sort_values(by='date')
CA_df_final.to_csv(input_data_dir+'California Agewise processed data.csv')
CA_df_final = CA_df_final.reset_index().drop(['date'],axis=1)

0-17 337
18-49 337
50-64 337
65+ 337


### Factor Calculation

In [10]:
CA_df_final

,Infected_0-17,Infected_18-49,Infected_50-64,Infected_65+
0,120,5302,2879,2342
1,137,5917,3253,2660
2,160,6610,3653,2966
3,173,7099,3884,3129
4,197,7753,4344,3516
...,...,...,...,...
332,447354,1993740,664406,371427
333,447770,1995109,664879,371704
334,448193,1996925,665595,372106
335,448653,1998780,666332,372554


In [11]:
CA_infected = CA_df_final.sum(axis=1)
CA_infected

0        10643
1        11967
2        13389
3        14285
4        15810
        ...   
332    3476927
333    3479462
334    3482819
335    3486319
336    3490972
Length: 337, dtype: int64

In [12]:
pop_California = 39512223
susceptible = pop_California - CA_infected
susceptible

0      39501580
1      39500256
2      39498834
3      39497938
4      39496413
         ...   
332    36035296
333    36032761
334    36029404
335    36025904
336    36021251
Length: 337, dtype: int64

In [13]:
CA_test_df = pd.read_csv(input_data_dir + "\California_statewide_testing.csv")

In [14]:
CA_test_df = CA_test_df.rolling(window = 7).mean()

CA_test_series = CA_test_df[-len(susceptible):].reset_index().drop(columns = ['index'])['tested']

In [15]:
CA_test_series

0      2.954457e+04
1      4.218671e+04
2      5.506486e+04
3      6.969943e+04
4      8.605200e+04
           ...     
332    4.805379e+07
333    4.825148e+07
334    4.844804e+07
335    4.863854e+07
336    4.882410e+07
Name: tested, Length: 337, dtype: float64

In [16]:
susceptible/CA_test_series

0      1337.016517
1       936.319803
2       717.314746
3       566.689553
4       458.983092
          ...     
332       0.749895
333       0.746770
334       0.743671
335       0.740686
336       0.737776
Length: 337, dtype: float64